In [196]:
import os
import requests
import pandas as pd
import re
import asyncio
from datetime import datetime, timedelta
from prophet import Prophet
from duckduckgo_search import DDGS
from langchain_community.llms import Ollama
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from bs4 import BeautifulSoup

In [199]:
#1. Getting USD to EGP for today only : 
headers = {
    "User-Agent": "Mozilla/5.0"  # required to avoid being blocked
}

def get_usd_price(_: str) -> str:
    url = "https://www.investing.com/currencies/usd-egp"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return f"❌ Failed to fetch data. Status code: {response.status_code}"

    soup = BeautifulSoup(response.content, "lxml")

    # Find the div containing the live price
    price_tag = soup.find("div", {"data-test": "instrument-price-last"})

    if not price_tag:
        return "❌ Could not find the exchange rate element."

    try:
        rate = float(price_tag.text.strip().replace(",", ""))

        # Save for Prophet forecast
        today = datetime.today().strftime("%Y-%m-%d")
        df = pd.DataFrame([{"ds": today, "y": rate}])
        df.to_csv("usd_egp_history.csv", index=False)

        return (
            f"Today ({today}):\n"
            f"1 USD = {rate:.2f} EGP\n"
            f"5 USD = {5 * rate:.2f} EGP\n"
            f"10 USD = {10 * rate:.2f} EGP\n"
            f"50 USD = {50 * rate:.2f} EGP\n"
            f"100 USD = {100 * rate:.2f} EGP\n"
        )
    except Exception as e:
        return f"❌ Parsing error: {str(e)}"

In [207]:
# 2. Forecast USD/EGP for tomorrow using Prophet
def forecast_usd(_: str) -> str:
    if not os.path.exists("usd_egp_price.csv"):
        raise FileNotFoundError("usd_egp_history.csv not found. Run get_usd_price() first.")

    df = pd.read_csv("usd_egp_price.csv")

    if len(df) < 2:
        yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
        df.loc[1] = [yesterday, df['y'][0] * 0.99]
        df = df.sort_values("ds")

    # Make sure ds column is datetime
    df["ds"] = pd.to_datetime(df["ds"])

    model = Prophet(daily_seasonality=True)
    model.fit(df)

    future = model.make_future_dataframe(periods=2)
    forecast = model.predict(future)

    # Ensure forecast dates are datetime
    forecast["ds"] = pd.to_datetime(forecast["ds"])

    # Get last actual rate (today)
    today_date = df["ds"].iloc[-1].date()
    today_rate = df["y"].iloc[-1]

    # Forecasted rates
    tomorrow_date = forecast.iloc[-2]["ds"].date()
    tomorrow_rate = forecast.iloc[-2]["yhat"]

    day_after_date = forecast.iloc[-1]["ds"].date()
    day_after_rate = forecast.iloc[-1]["yhat"]

    return (
        f"Today ({today_date}):\n"
        f"1 USD = {today_rate:.2f} EGP\n\n"
        f"Tomorrow ({tomorrow_date}):\n"
        f"1 USD = {tomorrow_rate:.2f} EGP\n\n"
        f"Day After Tomorrow ({day_after_date}):\n"
        f"1 USD = {day_after_rate:.2f} EGP"
    )

In [208]:
# 3. Let LLM explain the forecast in natural language
def explain_forecast(_: str) -> str:
    llm = Ollama(model="mistral-openorca")
    prompt = (
        "The model used historical exchange rates (USD to EGP) and applied time series forecasting "
        "to predict tomorrow's rate. It considers the trend and daily seasonality to generate an expected value and range. "
        "Now explain this forecast simply for a general user."
    )
    return llm.invoke(prompt)

In [209]:
# 4. إعداد الأدوات و الـ Agent
llm = Ollama(model="mistral-openorca", temperature=0)

In [210]:
tools = [
    Tool(name="Get USD Rate Today", func=get_usd_price, description="Scrapes today's USD to EGP exchange rate from the web."),
    Tool(name="Forecast USD Tomorrow", func=forecast_usd, description="Predicts the exchange rate for tomorrow using Prophet."),
    Tool(name="Explain Forecast", func=explain_forecast, description="Explains the forecast in simple terms using LLM.")
]

In [211]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [212]:
# 5. Run the agent
response = agent.run("Give me today's USD to EGP rate and a prediction for tomorrow with explanation.")
print("\n🤖 Agent Response:\n", response)



> Entering new AgentExecutor chain...
 To get the current exchange rate, I will use the "Get USD Rate Today" action. Then, I will predict the exchange rate for tomorrow using the "Forecast USD Tomorrow" action and explain it using the "Explain Forecast" action.

Action: Get USD Rate Today
Action Input: None
Observation: Today (2025-06-30):
1 USD = 49.70 EGP
5 USD = 248.50 EGP
10 USD = 497.00 EGP
50 USD = 2485.00 EGP
100 USD = 4970.00 EGP

Thought:

09:38:09 - cmdstanpy - INFO - Chain [1] start processing
09:38:09 - cmdstanpy - INFO - Chain [1] done processing


 To predict the exchange rate for tomorrow, I will use the "Forecast USD Tomorrow" action and explain it using the "Explain Forecast" action.

Action: Forecast USD Tomorrow
Action Input: None
Observation: Today (2025-06-30):
1 USD = 49.70 EGP

Tomorrow (2025-07-01):
1 USD = 50.20 EGP

Day After Tomorrow (2025-07-02):
1 USD = 50.69 EGP
 To explain the forecast, I will use the "Explain Forecast" action.

Action: Explain Forecast
Action Input: Tomorrow (2025-07-01)
Observation:  The model is like a smart guesser that uses information about past exchange rates (how much one type of money, like USD, is worth in another type, like EGP). It looks at how the rate has changed over time and tries to understand any patterns or trends it sees, such as if the rate usually goes up or down on certain days.

Then, the model makes a "best guess" about what the exchange rate will be tomorrow. This isn't an exact science because many things can affect exchange rates, like world events or changes in a cou